In [161]:
import gmaps
import pandas as pd
import requests
import json

# Google developer API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

In [162]:
# Creating weather stats dataframe from data in WeatherPy assignment
weather = pd.read_csv('../WeatherPy/retrieved_weather_data.csv')
weather.dropna(inplace = True)
weather.head()

,Name,Latitude (°),Longitude (°),Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
0,Aklavik,68.22,-135.01,35.60,93,4.70,90
1,Sistranda,63.73,8.83,53.60,93,11.41,75
2,Punta Arenas,-53.15,-70.92,42.39,81,12.75,98
3,Manakara,-22.13,48.02,64.31,89,7.02,0
4,Albany,42.60,-73.97,41.67,58,3.91,0


In [163]:
# Storing locations
locations = weather[["Latitude (°)", "Longitude (°)"]]
locations
lat_long = list(zip(locations['Latitude (°)'],locations['Longitude (°)']))
weather['Lat_Long'] = lat_long
weather.head(5)
lat_long

[(68.22, -135.01),
 (63.73, 8.83),
 (-53.15, -70.92),
 (-22.13, 48.02),
 (42.6, -73.97),
 (-46.19, 168.86),
 (72.79, -56.15),
 (-33.65, 115.33),
 (-28.65, 153.62),
 (5.84, 118.12),
 (50.6, 97.52),
 (68.75, 161.3),
 (59.7, 30.79),
 (37.37, 118.02),
 (65.28, -126.83),
 (-21.21, -159.78),
 (71.29, -156.79),
 (-20.23, 164.02),
 (-34.42, 19.23),
 (-35.34, 143.55),
 (70.47, -68.59),
 (35.73, 140.83),
 (24.76, 71.77),
 (-21.36, 55.77),
 (1.49, 124.85),
 (-46.6, 168.33),
 (-46.28, 169.85),
 (36.5, -5.75),
 (40.81, 114.88),
 (-24.79, -50.01),
 (-33.93, 18.42),
 (45.04, 41.97),
 (-35.35, 150.47),
 (-23.7, 133.88),
 (35.13, 140.3),
 (-2.57, 150.8),
 (-28.55, 16.43),
 (30.48, -115.95),
 (73.51, 80.55),
 (20.2, 1.01),
 (-33.59, 26.89),
 (62.46, -114.35),
 (-16.48, -151.75),
 (-20.41, 57.7),
 (42.1, -79.24),
 (-0.74, -90.35),
 (-23.12, -134.97),
 (62.27, 21.38),
 (-2.92, -39.92),
 (-31.43, 152.92),
 (-26.79, -60.44),
 (51.13, 93.6),
 (39.82, 109.98),
 (57.05, -135.33),
 (60.92, -46.05),
 (-9.8, -139

In [164]:
# Filtering for desired weather conditions
good_weather = weather.loc[(weather['Temperature (F)'] > 70) & (weather['Cloudiness (%)'] < 50) & (weather['Humidity (%)'] < 50)]
good_weather.head(5)

,Name,Latitude (°),Longitude (°),Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Lat_Long
32,Ulladulla,-35.35,150.47,79.05,37,16.11,20,"(-35.35, 150.47)"
39,Tessalit,20.20,1.01,87.42,16,8.66,0,"(20.2, 1.01)"
92,Pueblo,38.25,-104.61,73.47,33,4.70,1,"(38.25, -104.61)"
130,Kumano,34.34,132.59,74.28,49,6.93,20,"(34.34, 132.59)"
157,Makkah al Mukarramah,21.43,39.83,82.98,34,3.29,0,"(21.43, 39.83)"


In [165]:
# Adding column for nearest hotel
nearest_hotel = pd.DataFrame(good_weather)
nearest_hotel['Nearest_Hotel'] = ""
nearest_hotel.head(5)

,Name,Latitude (°),Longitude (°),Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Lat_Long,Nearest_Hotel
32,Ulladulla,-35.35,150.47,79.05,37,16.11,20,"(-35.35, 150.47)",
39,Tessalit,20.20,1.01,87.42,16,8.66,0,"(20.2, 1.01)",
92,Pueblo,38.25,-104.61,73.47,33,4.70,1,"(38.25, -104.61)",
130,Kumano,34.34,132.59,74.28,49,6.93,20,"(34.34, 132.59)",
157,Makkah al Mukarramah,21.43,39.83,82.98,34,3.29,0,"(21.43, 39.83)",


In [166]:
# params dictionary to update each iteration

params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

for index, row in nearest_hotel.iterrows():
    # get lat, lng from df
    lat = row["Latitude (°)"]
    lng = row["Longitude (°)"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    results = requests.get(base_url, params=params).json()    
    
    try:
        nearest_hotel.loc[index, "Nearest_Hotel"] = results["results"][0]["name"]
        print(f"Successfully added, ", results["results"][0]["name"])
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        continue

Successfully added,  Sandpiper Motel
Missing field/result... skipping.
Successfully added,  Hampton Inn & Suites Pueblo-Southgate
Successfully added,  ホテル コラソン
Successfully added,  Hotel Fairmont Makkah Clock Royal Tower
Successfully added,  Lamar Resort Abu Soma
Successfully added,  Shingu UI Hotel
Successfully added,  LOTTE HOTEL SEOUL
Successfully added,  Hotel pousada santo onofre
Successfully added,  ホテル松原屋
Successfully added,  Marriott's Ko Olina Beach Club
Successfully added,  San Ramon Marriott
Successfully added,  Hometowne Inn
Successfully added,  Batemans Bay Marina Resort
Successfully added,  Mövenpick Hotel Qassim
Successfully added,  Nag Aespen
Successfully added,  ibis Styles Mt Isa Verona
Successfully added,  The Esplanade Hotel
Successfully added,  Expeditions Ténére voyages
Successfully added,  ibis Styles Karratha
Successfully added,  Holiday Inn Express & Suites Elk Grove Central - Hwy 99
Successfully added,  Best Western Plus Winslow Inn
Successfully added,  Best W

In [168]:
nearest_hotel.head(5)

,Name,Latitude (°),Longitude (°),Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Lat_Long,Nearest_Hotel
32,Ulladulla,-35.35,150.47,79.05,37,16.11,20,"(-35.35, 150.47)",Sandpiper Motel
39,Tessalit,20.20,1.01,87.42,16,8.66,0,"(20.2, 1.01)",
92,Pueblo,38.25,-104.61,73.47,33,4.70,1,"(38.25, -104.61)",Hampton Inn & Suites Pueblo-Southgate
130,Kumano,34.34,132.59,74.28,49,6.93,20,"(34.34, 132.59)",ホテル コラソン
157,Makkah al Mukarramah,21.43,39.83,82.98,34,3.29,0,"(21.43, 39.83)",Hotel Fairmont Makkah Clock Royal Tower


In [169]:
# Dropping Tessalit since no hotel is found
nan_value = float("NaN")
nearest_hotel.replace("", nan_value, inplace=True)
nearest_hotel.dropna()

,Name,Latitude (°),Longitude (°),Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Lat_Long,Nearest_Hotel
32,Ulladulla,-35.35,150.47,79.05,37,16.11,20,"(-35.35, 150.47)",Sandpiper Motel
92,Pueblo,38.25,-104.61,73.47,33,4.70,1,"(38.25, -104.61)",Hampton Inn & Suites Pueblo-Southgate
130,Kumano,34.34,132.59,74.28,49,6.93,20,"(34.34, 132.59)",ホテル コラソン
157,Makkah al Mukarramah,21.43,39.83,82.98,34,3.29,0,"(21.43, 39.83)",Hotel Fairmont Makkah Clock Royal Tower
160,Safaga,26.73,33.94,86.00,45,11.41,0,"(26.73, 33.94)",Lamar Resort Abu Soma
186,Shingū,33.73,135.98,78.80,47,5.82,20,"(33.73, 135.98)",Shingu UI Hotel
203,Seoul,37.57,126.98,70.88,46,2.89,1,"(37.57, 126.98)",LOTTE HOTEL SEOUL
207,Parambu,-6.21,-40.69,75.49,42,5.55,12,"(-6.21, -40.69)",Hotel pousada santo onofre
217,Hikari,33.95,131.95,73.40,49,9.17,20,"(33.95, 131.95)",ホテル松原屋
247,Makakilo City,21.35,-158.09,84.29,48,9.17,40,"(21.35, -158.09)",Marriott's Ko Olina Beach Club


In [170]:
nearest_hotel['Country'] = ""
nearest_hotel.head(10)

,Name,Latitude (°),Longitude (°),Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Lat_Long,Nearest_Hotel,Country
32,Ulladulla,-35.35,150.47,79.05,37,16.11,20,"(-35.35, 150.47)",Sandpiper Motel,
39,Tessalit,20.20,1.01,87.42,16,8.66,0,"(20.2, 1.01)",NaN,
92,Pueblo,38.25,-104.61,73.47,33,4.70,1,"(38.25, -104.61)",Hampton Inn & Suites Pueblo-Southgate,
130,Kumano,34.34,132.59,74.28,49,6.93,20,"(34.34, 132.59)",ホテル コラソン,
157,Makkah al Mukarramah,21.43,39.83,82.98,34,3.29,0,"(21.43, 39.83)",Hotel Fairmont Makkah Clock Royal Tower,
160,Safaga,26.73,33.94,86.00,45,11.41,0,"(26.73, 33.94)",Lamar Resort Abu Soma,
186,Shingū,33.73,135.98,78.80,47,5.82,20,"(33.73, 135.98)",Shingu UI Hotel,
203,Seoul,37.57,126.98,70.88,46,2.89,1,"(37.57, 126.98)",LOTTE HOTEL SEOUL,
207,Parambu,-6.21,-40.69,75.49,42,5.55,12,"(-6.21, -40.69)",Hotel pousada santo onofre,
217,Hikari,33.95,131.95,73.40,49,9.17,20,"(33.95, 131.95)",ホテル松原屋,


In [171]:
params = {"key": gkey}

for index, row in nearest_hotel.iterrows():
    
    lat = row["Latitude (°)"]
    lng = row["Longitude (°)"]
    params["address"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    results = requests.get(base_url, params=params).json() 
    
    try:
        if "country" in results["types"]:
            nearest_hotel.loc[index, "Country"] = results["results"][0]["address_components"][5]["long_name"] 
            print(f"Successfully added, ", results["results"][0]["address_components"][5]["long_name"])

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        continue

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [172]:
nearest_hotel.head(10)

,Name,Latitude (°),Longitude (°),Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%),Lat_Long,Nearest_Hotel,Country
32,Ulladulla,-35.35,150.47,79.05,37,16.11,20,"(-35.35, 150.47)",Sandpiper Motel,
39,Tessalit,20.20,1.01,87.42,16,8.66,0,"(20.2, 1.01)",NaN,
92,Pueblo,38.25,-104.61,73.47,33,4.70,1,"(38.25, -104.61)",Hampton Inn & Suites Pueblo-Southgate,
130,Kumano,34.34,132.59,74.28,49,6.93,20,"(34.34, 132.59)",ホテル コラソン,
157,Makkah al Mukarramah,21.43,39.83,82.98,34,3.29,0,"(21.43, 39.83)",Hotel Fairmont Makkah Clock Royal Tower,
160,Safaga,26.73,33.94,86.00,45,11.41,0,"(26.73, 33.94)",Lamar Resort Abu Soma,
186,Shingū,33.73,135.98,78.80,47,5.82,20,"(33.73, 135.98)",Shingu UI Hotel,
203,Seoul,37.57,126.98,70.88,46,2.89,1,"(37.57, 126.98)",LOTTE HOTEL SEOUL,
207,Parambu,-6.21,-40.69,75.49,42,5.55,12,"(-6.21, -40.69)",Hotel pousada santo onofre,
217,Hikari,33.95,131.95,73.40,49,9.17,20,"(33.95, 131.95)",ホテル松原屋,


In [176]:
# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


# Humidity heat map
humidity = weather["Humidity (%)"].astype(float)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Getting coordinates of the hotels
nearest_lat_long = list(nearest_hotel['Lat_Long'])
nearest_lat_long

# Assigning the markers
markers = gmaps.marker_layer(nearest_lat_long)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…